# Credit Risk Resampling Techniques

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

# Read the CSV and Perform Basic Data Cleaning

In [3]:
columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [4]:
# Load the data
file_path = Path('C:/Users/racha\Class/Machine Learning Challenge/Module-17-Challenge-Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


# Split the Data into Training and Testing

In [5]:
# Create our features
X = df.drop(columns="loan_status")
X=pd.get_dummies(X)



In [6]:
Y = df['loan_status']
Y.shape

(68817,)

In [7]:
# Create X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y, 
                                                    random_state=1)
#Counter(X_train)
                                



# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [8]:
X_train.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,issue_d_Mar-2019,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
count,51612.000000,51612.000000,51612.000000,5.161200e+04,51612.000000,51612.000000,51612.000000,51612.000000,51612.000000,51612.000000,...,51612.000000,51612.0,51612.000000,51612.000000,51612.000000,51612.000000,51612.000000,51612.000000,51612.0,51612.0
mean,16711.575312,0.127699,481.504593,8.847633e+04,21.740295,0.218515,0.496241,12.582558,0.125804,17638.211734,...,0.176006,1.0,0.123634,0.876366,0.384465,0.615535,0.859839,0.140161,1.0,1.0
std,10291.735620,0.048201,288.491457,1.233078e+05,18.901909,0.715931,0.757929,6.016046,0.335870,21927.460755,...,0.380829,0.0,0.329167,0.329167,0.486473,0.486473,0.347158,0.347158,0.0,0.0
min,1000.000000,0.060000,30.890000,1.200000e+02,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.0
25%,9193.750000,0.088100,265.730000,5.015500e+04,13.870000,0.000000,0.000000,8.000000,0.000000,6296.000000,...,0.000000,1.0,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.0,1.0
50%,15000.000000,0.118000,404.790000,7.300000e+04,19.730000,0.000000,0.000000,11.000000,0.000000,12078.500000,...,0.000000,1.0,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.0,1.0
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.670000,0.000000,1.000000,16.000000,0.000000,21707.500000,...,0.000000,1.0,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.0,1.0
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,563245.000000,...,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0


In [9]:
y_train.describe()

count        51612
unique           2
top       low_risk
freq         51366
Name: loan_status, dtype: object

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=5)
Counter(y_train)

Counter({'low_risk': 51355, 'high_risk': 257})

In [11]:
Counter(X_train)

Counter({'loan_amnt': 1,
         'int_rate': 1,
         'installment': 1,
         'annual_inc': 1,
         'dti': 1,
         'delinq_2yrs': 1,
         'inq_last_6mths': 1,
         'open_acc': 1,
         'pub_rec': 1,
         'revol_bal': 1,
         'total_acc': 1,
         'out_prncp': 1,
         'out_prncp_inv': 1,
         'total_pymnt': 1,
         'total_pymnt_inv': 1,
         'total_rec_prncp': 1,
         'total_rec_int': 1,
         'total_rec_late_fee': 1,
         'recoveries': 1,
         'collection_recovery_fee': 1,
         'last_pymnt_amnt': 1,
         'collections_12_mths_ex_med': 1,
         'policy_code': 1,
         'acc_now_delinq': 1,
         'tot_coll_amt': 1,
         'tot_cur_bal': 1,
         'open_acc_6m': 1,
         'open_act_il': 1,
         'open_il_12m': 1,
         'open_il_24m': 1,
         'mths_since_rcnt_il': 1,
         'total_bal_il': 1,
         'il_util': 1,
         'open_rv_12m': 1,
         'open_rv_24m': 1,
         'max_bal_bc':

In [11]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)


In [13]:
X_resampled.head(5)

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,issue_d_Mar-2019,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
0,15000.0,0.2500,440.27,91000.0,37.92,0.0,0.0,18.0,0.0,33551.0,...,0,1,0,1,1,0,1,0,1,1
1,2000.0,0.1691,71.22,22000.0,4.96,0.0,0.0,12.0,0.0,2932.0,...,0,1,0,1,0,1,1,0,1,1
2,15000.0,0.1308,341.92,95000.0,14.10,0.0,1.0,11.0,0.0,10348.0,...,1,1,0,1,0,1,1,0,1,1
3,16000.0,0.1797,406.04,35000.0,42.70,0.0,0.0,10.0,0.0,1752.0,...,0,1,1,0,0,1,0,1,1,1
4,8000.0,0.0756,249.08,47000.0,17.16,0.0,1.0,15.0,0.0,6598.0,...,0,1,0,1,1,0,1,0,1,1


In [14]:
Counter(y_resampled)

Counter({0: 17532, 1: 4968})

Counter({0: 17532, 1: 4968})

In [15]:
#from sklearn.preprocessing import LabelEncoder
#le = LabelEncoder()
#df2 = X_resampled.copy()
#df2 = le.fit_transform(df2)

In [13]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=1, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [14]:
y_pred = model.predict(X_test)

In [15]:
y_pred

array(['low_risk', 'high_risk', 'low_risk', ..., 'high_risk', 'low_risk',
       'low_risk'], dtype=object)

In [16]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))
accuracy_score(y_test, y_pred)

0.6430688753269398


0.6430688753269398

In [17]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[   64,    26],
       [ 6115, 11000]], dtype=int64)

In [18]:
# Print the imbalanced classification report
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

   high_risk       0.01      0.71      0.02        90
    low_risk       1.00      0.64      0.78     17115

    accuracy                           0.64     17205
   macro avg       0.50      0.68      0.40     17205
weighted avg       0.99      0.64      0.78     17205



### SMOTE Oversampling

In [23]:
!pip install --upgrade imblearn

Requirement already up-to-date: imblearn in c:\users\racha\anaconda3\envs\pythondata\lib\site-packages (0.0)


In [24]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE
X_resampled, y_resampled = SMOTE(random_state=1).fit_resample(
   X_train, y_train
)

In [25]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

confusion_matrix(y_test, y_pred)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

   high_risk       0.01      0.63      0.02        90
    low_risk       1.00      0.67      0.80     17115

    accuracy                           0.67     17205
   macro avg       0.50      0.65      0.41     17205
weighted avg       0.99      0.67      0.79     17205



In [26]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
accuracy_score(y_test, y_pred)

0.6654460912525428

In [27]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[   57,    33],
       [ 5723, 11392]], dtype=int64)

In [28]:
# Print the imbalanced classification report
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

   high_risk       0.01      0.63      0.02        90
    low_risk       1.00      0.67      0.80     17115

    accuracy                           0.67     17205
   macro avg       0.50      0.65      0.41     17205
weighted avg       0.99      0.67      0.79     17205



# ClusterCentroids resampler

In [ ]:
# Resample the data using the ClusterCentroids resampler
from imblearn.under_sampling import ClusterCentroids
cc = ClusterCentroids(random_state=1)
X_resampled, y_resampled = cc.fit_resample(X_train, y_train)

In [30]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(solver='lbfgs', random_state=1)
classifier

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=1, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [31]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, y_pred)

0.6494741454864155

In [32]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[   57,    33],
       [ 5723, 11392]], dtype=int64)

In [33]:
# Print the imbalanced classification report
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

   high_risk       0.01      0.63      0.02        90
    low_risk       1.00      0.67      0.80     17115

    accuracy                           0.67     17205
   macro avg       0.50      0.65      0.41     17205
weighted avg       0.99      0.67      0.79     17205



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [20]:
# Resample the training data with SMOTEENN
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=0)
X_resampled, y_resampled = smote_enn.fit_resample(X, Y)

In [21]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=1, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [22]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, y_pred)

0.6769110916350181

In [23]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[  70,   20],
       [7414, 9701]], dtype=int64)

In [24]:
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.78      0.57      0.02      0.66      0.45        90
   low_risk       1.00      0.57      0.78      0.72      0.66      0.43     17115

avg / total       0.99      0.57      0.78      0.72      0.66      0.43     17205

